In [15]:
from typing import Optional
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset


class MyDataset(Dataset):
    def __init__(self, data, classes):
        super().__init__()
        self.data = torch.tensor(data, dtype=torch.float)
        self.classes = torch.tensor(classes, dtype=torch.long)

    def __getitem__(self, i):
        return self.data[i], self.classes[i]

    def __len__(self):
        return len(self.data)


class MyDataModule(pl.LightningDataModule):
    def __init__(self, train_path: str, train_classes_path: str, test_path: str, test_classes_path: str,
                 batch_size: int = 1):
        super().__init__()
        self.train_path = train_path
        self.train_classes_path = train_classes_path
        self.test_path = test_path
        self.test_classes_path = test_classes_path

        self.train_data = None
        self.test_data = None
        self.train_dataset = None
        self.test_dataset = None
        self.train_classes = None
        self.test_classes = None
        self.sep = ';'
        self.num_classes = None
        self.batch_size = batch_size
        self.vector_len = None

        self.prepare_data()

    @staticmethod
    def pad(text_tensor, total):
        n = total - len(text_tensor)
        return F.pad(text_tensor, (0, n))

    def prepare_data(self):
        self.download_dataset()
        self.num_classes = len(set(self.train_classes))

    def download_dataset(self):
        with open(self.train_path) as f:
            self.train_data = [[float(_) for _ in line.split(self.sep)] for line in f]
        with open(self.test_path) as f:
            self.test_data = [[float(_) for _ in line.split(self.sep)] for line in f]
        with open(self.train_classes_path) as f:
            self.train_classes = [int(line) - 1 for line in f]
        with open(self.test_classes_path) as f:
            self.test_classes = [int(line) - 1 for line in f]

    def setup(self, stage: Optional[str] = None):
        self.vector_len = self.count_vector_len()
        self.train_data = [i + [0] * (self.vector_len - len(i)) for i in self.train_data]
        self.test_data = [i + [0] * (self.vector_len - len(i)) for i in
                          list(map(lambda vector: vector[:self.vector_len], self.test_data))]
        self.train_dataset = MyDataset(data=self.train_data, classes=self.train_classes)
        self.test_dataset = MyDataset(data=self.test_data, classes=self.test_classes)

    def count_vector_len(self):
        max_len = 0
        for vector in self.train_data:
            max_len = max(max_len, len(vector))
        return max_len

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

    @property
    def num_inputs(self):
        if not self.vector_len:
            self.vector_len = self.count_vector_len()
        return self.vector_len

    @property
    def num_outputs(self):
        return self.num_classes

## Training SNN
- one LIF layer
- beta (decay rate) = 0.92
- spike threshold = 1
- potential subtract after spike emission

In [17]:
import torch
import torchmetrics
from torch import nn
import pytorch_lightning as pl
import snntorch as snn
import torch.nn.functional as F
from snntorch import surrogate


class SpikeText(pl.LightningModule):
    def __init__(self, num_inputs, num_hidden, beta, num_outputs, learning_rate):
        super().__init__()

        # Initialize layers
        self.spike_grad = surrogate.fast_sigmoid()
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta, spike_grad=self.spike_grad)
        self.fc2 = nn.Linear(num_hidden, num_outputs)
        # Initialize hidden states at t=0
        self.mem1 = self.lif1.init_leaky()
        self.lr = learning_rate
        # self.loss_fct = SF.mse_count_loss(correct_rate=0.8, incorrect_rate=0.2)
        self.loss_fct = F.cross_entropy
        self.train_accuracy = torchmetrics.Accuracy()
        self.test_accuracy = torchmetrics.Accuracy()

    def forward(self, x, mem1):
        cur1 = self.fc1(x)
        spk1, mem1_after = self.lif1(cur1, mem1)
        cur2 = self.fc2(spk1)
        return cur2, mem1_after

    def training_step(self, batch, batch_idx, mem1=None):
        if not mem1:
            mem1 = self.mem1
        x, y = batch
        cur, mem1_after = self.forward(x, mem1)
        loss = self.loss_fct(cur, y)
        self.log("train_loss", loss, prog_bar=True)
        self.train_accuracy(cur, y)
        self.log("train_acc", self.train_accuracy.compute(), prog_bar=True)
        return {'loss': loss, 'mem1': mem1_after}

    # def validation_step(self, batch, batch_idx):
    #     x, y = batch
    #     spk, mem = self.forward(x)
    #     loss = self.loss_fct(mem, y)
    #     self.log("val_loss", loss, prog_bar=True)
    #     return {'loss': loss}

    def test_step(self, batch, batch_idx, mem1=None):
        if not mem1:
            mem1 = self.mem1
        x, y = batch
        cur, mem1_after = self.forward(x, mem1)
        loss = self.loss_fct(cur, y)
        self.log("test_loss", loss, prog_bar=True)
        self.test_accuracy(cur, y)
        self.log("test_acc", self.test_accuracy.compute(), prog_bar=True)
        # self.test_balanced_accuracy(cur, y)
        # self.log("test_BA", self.test_balanced_accuracy.compute(), prog_bar=True)
        # self.test_balanced_accuracy(cur, y)
        # self.log("test_F1-score", self.test_f1_score.compute(), prog_bar=True)
        return {'loss': loss, 'mem1': mem1_after}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)


In [1]:
from pytorch_lightning.callbacks import ModelCheckpoint

ib_data_module = MyDataModule(
    train_path='data/processed/TitleMeshIB/TrainTitleIB.csv',
    test_path='data/processed/TitleMeshIB/TestTitleIB.csv',
    train_classes_path='data/processed/TitleMeshIB/TrainTitleIBClass.csv',
    test_classes_path='data/processedTitleMeshIB/TestTitleIBClass.csv'
)
b_data_module = MyDataModule(
    train_path='data/processed/TitleMeshB/TrainTitleB.csv',
    test_path='data/processed/TitleMeshB/TestTitleB.csv',
    train_classes_path='data/processed/TitleMeshB/TrainTitleBClass.csv',
    test_classes_path='data/processed/TitleMeshB/TestTitleBClass.csv'
)
ng_data_module = MyDataModule(
    train_path='data/processed/20NewsgroupShort/TrainShortBydate.csv',
    test_path='data/processed/20NewsgroupShort/TestShortBydate.csv',
    train_classes_path='data/processed/20NewsgroupShort/TrainShortBydateClass.csv',
    test_classes_path='data/processed/20NewsgroupShort/TestShortBydateClass.csv'
)

NameError: name 'MyDataModule' is not defined

In [19]:
model_ng = SpikeText(num_inputs=ng_data_module.num_inputs, num_hidden=1000, beta=0.92,
                     num_outputs=ng_data_module.num_outputs, learning_rate=1e-4)
model_ib = SpikeText(num_inputs=ib_data_module.num_inputs, num_hidden=1000, beta=0.92,
                     num_outputs=ib_data_module.num_outputs, learning_rate=1e-4)
model_b = SpikeText(num_inputs=b_data_module.num_inputs, num_hidden=1000, beta=0.92,
                    num_outputs=b_data_module.num_outputs, learning_rate=1e-4)

In [20]:
checkpoint_callback = ModelCheckpoint(
    monitor="train_acc",
    dirpath='lightning_logs',
    filename="{epoch:02d}-{train_acc:.2f}",
)

In [21]:
ib_trainer = pl.Trainer(default_root_dir='lightning_logs',
                        max_epochs=8, gpus=torch.cuda.device_count(),
                        callbacks=[checkpoint_callback])
b_trainer = pl.Trainer(default_root_dir='lightning_logs',
                       max_epochs=8, gpus=torch.cuda.device_count(),
                       callbacks=[checkpoint_callback])
ng_trainer = pl.Trainer(default_root_dir='lightning_logs',
                        max_epochs=8, gpus=torch.cuda.device_count(),
                        callbacks=[checkpoint_callback])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [15]:
training_sets = [(model_ng, ng_data_module, ng_trainer), (model_b, b_data_module, b_trainer),
                 (model_ib, ib_data_module, ib_trainer)]
for model, data_module, trainer in training_sets:
    trainer.fit(model, datamodule=data_module)


  | Name           | Type     | Params
--------------------------------------------
0 | fc1            | Linear   | 752 K 
1 | lif1           | Leaky    | 0     
2 | fc2            | Linear   | 20.0 K
3 | train_accuracy | Accuracy | 0     
4 | test_accuracy  | Accuracy | 0     
--------------------------------------------
772 K     Trainable params
0         Non-trainable params
772 K     Total params
3.088     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]


  | Name           | Type     | Params
--------------------------------------------
0 | fc1            | Linear   | 1.6 M 
1 | lif1           | Leaky    | 0     
2 | fc2            | Linear   | 15.0 K
3 | train_accuracy | Accuracy | 0     
4 | test_accuracy  | Accuracy | 0     
--------------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.484     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]


  | Name           | Type     | Params
--------------------------------------------
0 | fc1            | Linear   | 1.6 M 
1 | lif1           | Leaky    | 0     
2 | fc2            | Linear   | 15.0 K
3 | train_accuracy | Accuracy | 0     
4 | test_accuracy  | Accuracy | 0     
--------------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.484     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [16]:
for model, data_module, trainer in training_sets:
    trainer.test(model, datamodule=data_module)

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.23975901305675507, 'test_loss': 3.216249465942383}
--------------------------------------------------------------------------------


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.3863840699195862, 'test_loss': 2.1964197158813477}
--------------------------------------------------------------------------------


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.3924439251422882, 'test_loss': 2.207373857498169}
--------------------------------------------------------------------------------


## Training and testing MLP network:
- two layer
- sigmoid activation function

In [54]:
class Perceptron(pl.LightningModule):
    def __init__(self, num_inputs, num_hidden, num_outputs, learning_rate, no_class):
        super().__init__()
        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.fc2 = nn.Linear(num_hidden, num_outputs)
        self.lr = learning_rate
        self.loss_fct = F.cross_entropy
        self.train_accuracy = torchmetrics.Accuracy()
        self.test_accuracy = torchmetrics.Accuracy()
        # self.test_balanced_accuracy = torchmetrics.Accuracy(average='micro', num_classes=no_class)
        # self.test_f1_score = torchmetrics.F1Score(average='macro', num_classes=no_class);
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        cur1 = self.fc1(x)
        cur1 = self.sigmoid(cur1)
        cur2 = self.fc2(cur1)
        cur2 = self.sigmoid(cur2)
        return cur2

    def training_step(self, batch, batch_idx):
        x, y = batch
        cur = self.forward(x)
        loss = self.loss_fct(cur, y)
        self.log("train_loss", loss, prog_bar=True)
        self.train_accuracy(cur, y)
        self.log("train_acc", self.train_accuracy.compute(), prog_bar=True)
        return {'loss': loss}

    def test_step(self, batch, batch_idx):
        x, y = batch
        cur = self.forward(x)
        loss = self.loss_fct(cur, y)
        self.log("test_loss", loss, prog_bar=True)
        self.test_accuracy(cur, y)
        self.log("test_acc", self.test_accuracy.compute(), prog_bar=True)
        # self.test_balanced_accuracy(cur, y)
        # self.log("test_BA", self.test_balanced_accuracy.compute(), prog_bar=True)
        # self.test_f1_score(cur, y)
        # self.log("test_F1-score", self.test_f1_score.compute(), prog_bar=True)
        return {'loss': loss}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

In [16]:
ib_data_module = MyDataModule(
    train_path='data/processed/TitleMeshIB/TrainTitleIB.csv',
    test_path='data/processed/TitleMeshIB/TestTitleIB.csv',
    train_classes_path='data/processed/TitleMeshIB/TrainTitleIBClass.csv',
    test_classes_path='data/processedTitleMeshIB/TestTitleIBClass.csv'
)
b_data_module = MyDataModule(
    train_path='data/processed/TitleMeshB/TrainTitleB.csv',
    test_path='data/processed/TitleMeshB/TestTitleB.csv',
    train_classes_path='data/processed/TitleMeshB/TrainTitleBClass.csv',
    test_classes_path='data/processed/TitleMeshB/TestTitleBClass.csv'
)
ng_data_module = MyDataModule(
    train_path='data/processed/20NewsgroupShort/TrainShortBydate.csv',
    test_path='data/processed/20NewsgroupShort/TestShortBydate.csv',
    train_classes_path='data/processed/20NewsgroupShort/TrainShortBydateClass.csv',
    test_classes_path='data/processed/20NewsgroupShort/TestShortBydateClass.csv'
)

In [56]:
ng_mlp_model = Perceptron(num_inputs=ng_data_module.num_inputs, num_hidden=1000,
                          num_outputs=ng_data_module.num_outputs, learning_rate=1e-4,
                          no_class=ng_data_module.num_classes)
ib_mlp_model = Perceptron(num_inputs=ib_data_module.num_inputs, num_hidden=1000,
                          num_outputs=ib_data_module.num_outputs, learning_rate=1e-4,
                          no_class=ib_data_module.num_classes)
b_mlp_model = Perceptron(num_inputs=b_data_module.num_inputs, num_hidden=1000,
                         num_outputs=b_data_module.num_outputs, learning_rate=1e-4, no_class=b_data_module.num_classes)

checkpoint_callback = ModelCheckpoint(
    monitor="train_acc",
    dirpath='lightning_logs',
    filename="{epoch:02d}-{train_acc:.2f}",
)
mlp_trainer_ib = pl.Trainer(default_root_dir='lightning_logs',
                            max_epochs=8, gpus=torch.cuda.device_count(),
                            callbacks=[checkpoint_callback])
mlp_trainer_b = pl.Trainer(default_root_dir='lightning_logs',
                           max_epochs=8, gpus=torch.cuda.device_count(),
                           callbacks=[checkpoint_callback])
mlp_trainer_ng = pl.Trainer(default_root_dir='lightning_logs',
                            max_epochs=8, gpus=torch.cuda.device_count(),
                            callbacks=[checkpoint_callback])

mlp_training_sets = [(ng_mlp_model, ng_data_module, mlp_trainer_ng), (ib_mlp_model, ib_data_module, mlp_trainer_ib),
                     (b_mlp_model, b_data_module, mlp_trainer_b)]

for mlp_model, mlp_data_module, mlp_trainer in mlp_training_sets:
    mlp_trainer.fit(mlp_model, datamodule=mlp_data_module)
    mlp_trainer.test(mlp_model, datamodule=mlp_data_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name           | Type     | Params
--------------------------------------------
0 | fc1            | Linear   | 752 K 
1 | fc2            | Linear   | 20.0 K
2 | train_accuracy | Accuracy | 0     
3 | test_accuracy  | Accuracy | 0     
4 | sigmoid        | Sigmoid  | 0     
--------------------------------------------
772 K     Trainable params
0         Non-trainable params
772 K     Total params
3.088     Total estimated model params size (MB)
/Users/piotr/praca/nauka/publikacje/Hierarchical Text Classification/Software/SNNTorch-short-text-classifier/venv/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Chec

Training: 0it [00:00, ?it/s]

/Users/piotr/praca/nauka/publikacje/Hierarchical Text Classification/Software/SNNTorch-short-text-classifier/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.005375939887017012, 'test_loss': 3.132855176925659}
--------------------------------------------------------------------------------


...

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.31856489181518555, 'test_loss': 2.401262044906616}
--------------------------------------------------------------------------------



  | Name           | Type     | Params
--------------------------------------------
0 | fc1            | Linear   | 1.6 M 
1 | fc2            | Linear   | 15.0 K
2 | train_accuracy | Accuracy | 0     
3 | test_accuracy  | Accuracy | 0     
4 | sigmoid        | Sigmoid  | 0     
--------------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.484     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.30712130665779114, 'test_loss': 2.400636672973633}
--------------------------------------------------------------------------------


## Training and testing decision tree
* split crititerion - Gini index
* max depth of tree - none

In [17]:
from sklearn import tree
from sklearn.metrics import accuracy_score

for data_module in [ng_data_module, ib_data_module, b_data_module]:
    X = data_module.train_data
    y = data_module.train_classes
    clf = tree.DecisionTreeClassifier()
    clf.fit(X, y)
    pred = clf.predict(data_module.test_data)
    print("RESULT ACC:", accuracy_score(data_module.test_classes, pred))

    # print(classification_report(data_module.test_classes, pred))


RESULT ACC: 0.4374668082846522
RESULT ACC: 0.2614879649890591
RESULT ACC: 0.25483870967741934


## Training and testing logistic regression
- max_iter = 100

In [19]:
from sklearn.linear_model import LogisticRegressionCV

for data_module in [ng_data_module, ib_data_module, b_data_module]:
    X = data_module.train_data
    y = data_module.train_classes
    lrcv = LogisticRegressionCV()
    lrcv.fit(X, y)
    lrcv_pred = lrcv.predict(data_module.test_data)
    print("RESULT:", accuracy_score(data_module.test_classes, lrcv_pred))

/Users/piotr/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/piotr/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

RESULT: 0.5912108337758896


/Users/piotr/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/piotr/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

RESULT: 0.4026258205689278


/Users/piotr/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/piotr/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

RESULT: 0.4032258064516129


## Training and testing SVC
* kernel = RBF
* C = 1

In [4]:
from sklearn import svm
from sklearn.metrics import accuracy_score

for data_module in [ng_data_module, ib_data_module, b_data_module]:
    X = data_module.train_data
    y = data_module.train_classes
    svc = svm.SVC()
    svc.fit(X, y)
    svc_pred = svc.predict(data_module.test_data)
    print("RESULT:", accuracy_score(data_module.test_classes, svc_pred))


RESULT: 0.6104620286776421
RESULT: 0.3665207877461707
RESULT: 0.36989247311827955
